# P2SH MultiSig BIP322
## Role: Creator / Updator (see role definitions [here](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawiki#Roles))

## Workflow diagram

Taken from [BIP174: Partially Signed Bitcoin Transaction Format](https://github.com/bitcoin/bips/blob/master/bip-0174.mediawik)

<img src="https://github.com/bitcoin/bips/raw/master/bip-0174/multisig-workflow.svg">


In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Create HD Key from mnemonic

Note: this would typically access some wallet

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
mnemonic = 'priority orange artist purpose sport blade spike leg calm renew dress figure session play solve fatal spy reform humble carry finger response order print'

In [4]:
hd_rootkey = HDPrivateKey.from_mnemonic(mnemonic)

root_fp = hd_rootkey.fingerprint().hex()

## 2. Select Key to Sign With

In [5]:
bip32_path = "m/45'/0/0/1"

hd_priv_a = hd_rootkey.traverse(bip32_path)
pubkey = hd_priv_a.pub.point


## 3. Fetch Other Signing Parties Public Keys

See [signer_1](./Signer_1.ipynb) and [signer_2](./Signer_2.ipynb) notebooks steps 1 to 3

In [6]:
signer_1_pubkey = '02fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b'
signer_2_pubkey = '02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92'


## 4. Create 2-of-3 MultiSig Pay-to-Script Hash

Note: This p2sh_address would likely be something that the signers would want to include in a DID Document.

In [7]:
from buidl.script import RedeemScript

# YOU COULD CHANGE THIS
quorum_m=2

redeem_script = RedeemScript.create_p2sh_multisig(
            quorum_m=quorum_m,
            pubkey_hexes=[pubkey.sec().hex(),signer_1_pubkey,signer_2_pubkey],
            sort_keys=False,
        )


script_pubkey = redeem_script.script_pubkey()
p2sh_address = redeem_script.address()

In [8]:
print("The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address : ", p2sh_address)

The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address :  3LnYoUkFrhyYP3V7rq3mhpwALz1XbCY9Uq


## 5. Create the virtual to_spend transaction locked with the p2sh_multisig

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

The message_challenge is the above 2-3 p2sh scriptPubKey

In [9]:
from src.message import create_to_spend_tx

## You could change this
message_to_sign = "This will be a p2sh 2-of-3 multisig BIP 322 signed message"

to_spend_tx = create_to_spend_tx(p2sh_address, message_to_sign)

## 6. Create PSBT to_sign transaction

In [55]:
from src.message import create_to_sign_tx

to_sign_tx = create_to_sign_tx(to_spend_tx.hash())

# NOTE: This is a bug in the current create_to_sign_tx function
# It SHOULD be able to know if it is a segwit transaction or not
to_sign_tx.segwit = False

In [56]:
from buidl.psbt_helper import create_multisig_psbt
from buidl.psbt import PSBT

fee = 0
# psbt_to_sign_bip322 = create_multisig_psbt(pubkey_records,input_dicts,output_dicts,fee)
psbt_to_sign_bip322 = PSBT.create(to_sign_tx)

In [57]:
psbt_to_sign_bip322.validate()

True

## 7. Update PSBT with data know by Creator

This includes:
    
- A lookup for the to_spend_tx referenced as an input to the to_sign_tx
- A lookup for creators HD Public key that is part of the p2sh multisig
- A lookup for the redeemscript associated with the p2sh multsig stored in the scriptPubKey of the TxOut for the to_spend_tx

### 7.1 Define the tx lookup

This is a map of a transaction has to the actual transaction object

In [61]:
tx_lookup = {}

tx_lookup[to_spend_tx.hash()] = to_spend_tx

### 7.2 Define a pubkey_lookup

This maps from the sec representation of a S256Point to a NamedHDPublicKey object.

Note: a NamedHDPublicKey is a HDPublicKey with additioanl raw path data for the fingerprint of the root HDPublicKey and the derivation path from that root.

In [68]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=hd_priv_a.pub,
    xfp_hex=root_fp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

### 7.3 Define redeem_lookup

This is a map from a hash160 of a redeem script to the actual redeem script

In [63]:
redeem_lookup = {}
redeem_lookup[redeem_script.hash160()] = redeem_script

In [70]:
# This is not set until after the update
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

OP_HASH160 d1763eda828dbe7ae8374d2d7fb6c3381970b272 OP_EQUAL 

In [64]:
psbt_to_sign_bip322.update(tx_lookup,pubkey_lookup,redeem_lookup)

In [69]:
# As you can see update populates the scriptPubKey from the to_spend transaction into the tx_obj in the PSBT 
# (the to_sign obj)
psbt_to_sign_bip322.tx_obj.tx_ins[0]._script_pubkey

OP_HASH160 d1763eda828dbe7ae8374d2d7fb6c3381970b272 OP_EQUAL 

## 8. Serialize and Share PSBT with Other Signing Parties

See 

In [73]:
from io import BytesIO

b64_psbt = psbt_to_sign_bip322.serialize_base64()

print("Copy this vvv \n\n")

print(f"b64_psbt = '{b64_psbt}'")

Copy this vvv 


b64_psbt = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAAAEEaVIhAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeIQL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCyEC+uXB9l+Tuy7o40ywy9pfD8ctGNlFEffbtagn43c+3ZJTriIGAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeFHsF6hstAACAAAAAAAAAAAABAAAAAAA='


## 9. Recieve Signed PSBTs from Signer 1 and Signer 2 Notebooks

Note: in this case the 2-3 multisig is satisfied from signatures provided by signer_1 and signer_2. The Creator has no contributed a signature.

You could easily change this notebook so that it is the creator's signature plus one of the signers that satisfy the p2sh multisig instead though.

In [88]:
# Copy this from signer_1.ipynb

signer_1_psbt_b64 = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAACICAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLRzBEAiEAnqF2PrW8sM/5oVrfSpRlCatsF8TSSI45ti+3meo9LGwCH0is4oXB+V89FRxD/l7RESR0UdU/YYZ1SwyAN636w2UBAQRpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuIgYCKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4UewXqGy0AAIAAAAAAAAAAAAEAAAAiBgL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCxQAAAAALQAAgAAAAAAAAAAAAwAAAAAA'

In [ ]:
signer_1_psbt = PSBT.parse_base64(signer_1_psbt_b64)

In [86]:
# Copy this from signer_2.ipynb

signer_2_psbt_b64 = 'cHNidP8BAD0AAAAAAc1x8do1+Xx//5kLaBOvxoX7QqCmliMoz3h6Bo2b+7VkAAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAACICAvrlwfZfk7su6ONMsMvaXw/HLRjZRRH327WoJ+N3Pt2SRzBEAiAfd5b9XIUOhe8Jqx3yrGDCPk0NWjTB5e8zqrIZcsJHlgIgTCNsRXp9LpFY/pCwo5wzC+UavX9KB0ILqlbh4fzv9iABAQRpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuIgYCKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4UewXqGy0AAIAAAAAAAAAAAAEAAAAiBgL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dkhQAHQ2eLQAAgAAAAAAAAAAABgAAAAAA'


In [89]:
signer_2_psbt = PSBT.parse_base64(signer_2_psbt_b64)

## 10. Combine PSBTs 

In [92]:
# Before combination the psbt only has a single sig
len(signer_1_psbt.psbt_ins[0].sigs)

2

In [90]:
signer_1_psbt.combine(signer_2_psbt)

In [93]:
# Afterwards there are two signatures
len(signer_1_psbt.psbt_ins[0].sigs)

2

## 11. Finalize the PSBT to produce the scriptSig on the PSBT Input from the sigs

In [ ]:
# ScriptSig not populated until after finalization
signer_1_psbt.psbt_ins[0].script_sig

In [94]:
signer_1_psbt.finalize()

signer_1_psbt.psbt_ins[0].script_sig

## 12. Create Final Bitcoin Tx Object from PSBT

In [99]:
# Remember: currently the final_tx() function on the PSBT fails because there are no funds for a tx fee
# BUT for a bip322 we don't need any fee
signer_1_psbt.final_tx()

0

In [100]:
## Proposed update to the final_tx function in psbt.py

def final_tx(psbt, is_bip322=False):
        """Returns the broadcast-able transaction"""
        # clone the transaction from self.tx_obj
        tx_obj = psbt.tx_obj.clone()
        # determine if the transaction is segwit by looking for a witness field
        #  in any PSBTIn. if so, set tx_obj.segwit = True
        if any([psbt_in.witness for psbt_in in psbt.psbt_ins]):
            tx_obj.segwit = True
        # iterate through the transaction and PSBT inputs together
        #  using zip(tx_obj.tx_ins, self.psbt_ins)
        for tx_in, psbt_in in zip(tx_obj.tx_ins, psbt.psbt_ins):
            # set the ScriptSig of the transaction input
            tx_in.script_sig = psbt_in.script_sig
            # Exercise 7: if the tx is segwit, set the witness as well
            if tx_obj.segwit:
                # witness should be the PSBTIn witness or an empty Witness()
                tx_in.witness = psbt_in.witness or Witness()
        # check to see that the transaction verifies
        if not is_bip322 and not tx_obj.verify():
            raise RuntimeError("transaction invalid")
        # return the now filled in transaction
        return tx_obj

In [101]:
final_to_sign_tx = final_tx(signer_1_psbt, True)

## 13. Check Signature on Input is Valid

In [103]:
final_to_sign_tx.verify_input(0)

True

## 14. Serialize to_sign transaction as BIP322 Signature

In [106]:
from buidl.helper import base64_encode
bip322_musig = base64_encode(final_to_sign_tx.serialize())

print(bip322_musig)

AAAAAAHNcfHaNfl8f/+ZC2gTr8aF+0KgppYjKM94egaNm/u1ZAAAAAD8AEcwRAIhAJ6hdj61vLDP+aFa30qUZQmrbBfE0kiOObYvt5nqPSxsAh9IrOKFwflfPRUcQ/5e0REkdFHVP2GGdUsMgDet+sNlAUcwRAIgH3eW/VyFDoXvCasd8qxgwj5NDVo0weXvM6qyGXLCR5YCIEwjbEV6fS6RWP6QsKOcMwvlGr1/SgdCC6pW4eH87/YgAUxpUiECKJfGy28imLcuAeNBLHCNv3NRP5jnJwFDNRXCYNY/vJ4hAv1RQtaZs7+vKqQeWl2rb/jd/gMxkEjUnjZdDGPDZkMLIQL65cH2X5O7LujjTLDL2l8Pxy0Y2UUR99u1qCfjdz7dklOuAAAAAAEAAAAAAAAAAAFqAAAAAA==


## 15. Verify BIP322 Signature against message and address

In [107]:
from src.message import verify_message

In [109]:
verify_message(p2sh_address, bip322_musig, message_to_sign)

True